<a href="https://colab.research.google.com/github/sunnnymskang/NIST_BLE_challenge/blob/master/1DConvNet_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!unzip /content/gdrive/My\ Drive/tc4tl2.zip

Streaming output truncated to the last 5000 lines.
  inflating: tc4tl/data/test/dtqgkzal_tc4tl20.csv  
  inflating: tc4tl/data/test/deyrphuq_tc4tl20.csv  
  inflating: tc4tl/data/test/ldeulecu_tc4tl20.csv  
  inflating: tc4tl/data/test/vyrtaosl_tc4tl20.csv  
  inflating: tc4tl/data/test/vhmftaib_tc4tl20.csv  
  inflating: tc4tl/data/test/enfnsfuk_tc4tl20.csv  
  inflating: tc4tl/data/test/vqluoklg_tc4tl20.csv  
  inflating: tc4tl/data/test/wukowems_tc4tl20.csv  
  inflating: tc4tl/data/test/icemaeom_tc4tl20.csv  
  inflating: tc4tl/data/test/ghxodajt_tc4tl20.csv  
  inflating: tc4tl/data/test/sqkppypq_tc4tl20.csv  
  inflating: tc4tl/data/test/cxjgoseh_tc4tl20.csv  
  inflating: tc4tl/data/test/peivgvnm_tc4tl20.csv  
  inflating: tc4tl/data/test/voyejzox_tc4tl20.csv  
  inflating: tc4tl/data/test/qtabqnsc_tc4tl20.csv  
  inflating: tc4tl/data/test/teyrezur_tc4tl20.csv  
  inflating: tc4tl/data/test/fhebovpi_tc4tl20.csv  
  inflating: tc4tl/data/test/exkugnde_tc4tl20.csv  
  inflating: 

In [ ]:
import statistics
from sklearn.model_selection import train_test_split
import os
import random
import math
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(0)
import numpy as np
np.random.seed(0)
random.seed(0)


NUM_READINGS_PER_INTERVAL = 150
INTERVAL_LENGTH = 4  # length in seconds of each interval


def CSELoss(predictions, targets, epsilon=1e-12):
    predictions = torch.clamp(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -torch.sum(targets*torch.log(predictions+1e-9))/N
    return ce


print(CSELoss(torch.Tensor([[0.25,0.25,0.25,0.25],
                        [0.01,0.01,0.01,0.96]]), torch.Tensor([[0,0,0,1],
                  [0,0,0,1]])))

def load_data(key_path, data_path):
    # first pass to find the various values for the categorical features to use for one-hot encoding
    fixed_variables_possible_values = [set() for _ in range(9)]
    class_labels_possible_values = set()
    for file_id in os.listdir(data_path):
        if file_id.startswith("."):
            # there are some weird extra files starting with .
            continue
        with open(os.path.join(data_path, file_id), 'r', errors="ignore") as data_file:
            for index in range(7):
                value = data_file.readline().strip().split(",")[1]
                fixed_variables_possible_values[index].add(value)
    with open(key_path, 'r', errors="ignore") as key_file:
        key_file.readline()  # skip header
        for line in key_file:
            record = line.split("\t")
            transmitter_position, receiver_position = record[1].split("_")
            if len(record) == 5:
                # has labels
                class_labels_possible_values.add(float(record[2]))
            fixed_variables_possible_values[7].add(transmitter_position)
            fixed_variables_possible_values[8].add(receiver_position)
    fixed_variables_possible_values = [list(fixed_variable_possible_values) for fixed_variable_possible_values in fixed_variables_possible_values]
    class_labels_possible_values = list(class_labels_possible_values)
    X = list()
    y = list()
    tensor_to_file = list()
    with open(key_path, 'r') as key_file:
        key_file.readline()  # skip header
        for line in key_file:
            record = line.split("\t")
            file_id = record[0]
            with open(os.path.join(data_path, file_id)) as data_file:
                # fixed variables tx_device, tx_power, rx_device, tx_carry, rx_carry, rx_pose, tx_pose,
                #                               transmitter_position, receiver_position
                fixed_variables = [data_file.readline().strip().split(",")[1] for _ in range(7)]
                fixed_variables.extend(record[1].split("_"))
                fixed_part = list()
                for variable, key in zip(fixed_variables, fixed_variables_possible_values):
                    fixed_part.extend([int(possible_value == variable) for possible_value in key])
                print("Loading file {} with fixed variables of {}".format(file_id, fixed_part))
                interval_start_time = 0
                interval_data = list()
                num_intervals = 0
                reading_count = 0
                previous_value = {
                    "Bluetooth": (0,),
                    "Accelerometer": (0,0,0),
                    "Gyroscope": (0,0,0),
                    "Altitude": (0,0),
                    "Attitude": (0,0,0),
                    "Activity": (0,0),
                    "Gravity": (0,0,0),
                    "Magnetic-field": (0,0,0),
                    "Heading": (0,0,0,0)
                }
                for line in data_file:
                    reading = line.strip().split(",")
                    curr_time = float(reading[0])
                    if (curr_time - interval_start_time) > INTERVAL_LENGTH:
                        if reading_count > NUM_READINGS_PER_INTERVAL:
                            # randomly remove readings
                            for _ in range(reading_count - NUM_READINGS_PER_INTERVAL):
                                interval_data.pop(math.floor(random.random() * len(interval_data)))
                        else:
                            # randomly duplicate readings
                            # todo: try other methods such as averaging
                            for _ in range(NUM_READINGS_PER_INTERVAL - reading_count):
                                random_index = math.floor(random.random() * len(interval_data))
                                interval_data.insert(random_index, interval_data[random_index])
                        X.append(interval_data)
                        num_intervals += 1

                        # reset values
                        interval_start_time = curr_time
                        reading_count = 0
                        interval_data = list()
                        previous_value = {
                            "Bluetooth": (0,),
                            "Accelerometer": (0,0,0),
                            "Gyroscope": (0,0,0),
                            "Altitude": (0,0),
                            "Attitude": (0,0,0),
                            "Activity": (0,0),
                            "Gravity": (0,0,0),
                            "Magnetic-field": (0,0,0),
                            "Heading": (0,0,0,0)
                        }
                    type = reading[1]
                    if type == "Bluetooth":
                        previous_value[type] = (float(reading[2]), )
                    elif type == "Heading":
                        previous_value[type] = (float(reading[2]), float(reading[3]), float(reading[4]),
                                                float(reading[5]))
                    elif type == "Altitude":
                        previous_value[type] = (float(reading[2]), float(reading[3]))
                    elif type == "Activity":
                        # todo: add one-hot encoding for reading[3]
                        previous_value[type] = (float(reading[2]), float(reading[4]))
                    else:
                        previous_value[type] = (float(reading[2]), float(reading[3]), float(reading[4]))
                    # combine the values in previous_value into one giant list
                    interval_data.append([reading for value in previous_value.values() for reading in value])# + fixed_part)
                    reading_count += 1
                # the last interval needs to be added manually
                if reading_count > NUM_READINGS_PER_INTERVAL:
                    # randomly remove readings
                    for i in range(reading_count - NUM_READINGS_PER_INTERVAL):
                        interval_data.pop(math.floor(random.random() * len(interval_data)))
                else:
                    # randomly duplicate readings
                    # todo: try other methods such as averaging
                    for i in range(NUM_READINGS_PER_INTERVAL - reading_count):
                        random_index = math.floor(random.random() * len(interval_data))
                        interval_data.insert(random_index, interval_data[random_index])
                X.append(interval_data)
                num_intervals += 1

            if len(record) == 5:
                # this means this file has labels
                distance = float(record[2])
                label = torch.zeros(INTERVAL_LENGTH)
                label[class_labels_possible_values.index(distance)] = 1
                for _ in range(num_intervals):
                    y.append(label)
                    
            for _ in range(num_intervals):
                tensor_to_file.append(file_id)

            # good for debugging
            # if len(X) > 105:
            #     break
    return [torch.Tensor(interval) for interval in X], y, class_labels_possible_values, tensor_to_file


def eval_model(model, val_data_loader, save=True):
  model.eval()
  with torch.no_grad():
      test_total_loss = set()
      test_total_acc = set()
      test_total_tc_acc = set()
      for idx, batch in enumerate(val_data_loader):
          input = batch[0]
          label = torch.max(batch[1],axis=1)[1]
          prediction = model(input, batch_size=len(label))
          loss = loss_fn(prediction, batch[1])
          prediction = torch.max(prediction, 1)[1].view(label.size())
          num_corrects = (prediction == label).float().sum()
          acc = 100.0 * num_corrects / len(label)
          prediction = ((prediction == 0) + (prediction == 3).float())
          label = ((label == 0) + (label == 3).float())
          num_tc_corrects = (label == prediction).float().sum()
          tc_acc = 100.0 * num_tc_corrects / len(label)
          test_total_loss.add(loss.item())
          test_total_acc.add(acc.item())
          test_total_tc_acc.add(tc_acc.item())
      curr_acc = statistics.mean(test_total_acc)
      if best_acc < curr_acc and save:
        torch.save(model.state_dict(), MODEL_PATH)
      return statistics.mean(test_total_loss), statistics.mean(test_total_acc), statistics.mean(test_total_tc_acc)
import json

def output_predictions(model, X, labels_to_distance, intervals_to_file, output_path):
  model.eval()
  print(list(intervals_to_file))
  with torch.no_grad():
      with open(output_path, "w") as f:
        f.write("fileid\tdistance\n")
        file_to_interval_pred = dict()
        i = 0
        for tensor, file_id in zip(X, intervals_to_file):
            i+=1
            print(i)
            if file_id not in file_to_interval_pred:
              file_to_interval_pred[file_id] = list()            
            input = tensor.view(1, tensor.shape[0], tensor.shape[1])
            prediction = model(input, batch_size=1)
            prediction = labels_to_distance[torch.max(prediction, 1)[1]]
            file_to_interval_pred[file_id].append(str(prediction))
        file_to_pred = [file_id + "\t" + max(set(preds), key=preds.count) for file_id, preds in file_to_interval_pred.items()]
        file_to_pred.sort()
        f.write("\n".join(file_to_pred))
  return

tensor(0.7136)


In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers=1):
        super(GRU, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        # GRU Layer
        self.gru = nn.GRU(self.input_dim, self.hidden_dim, num_layers=num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, 128)
        self.linear2 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout()

    def forward(self, input, batch_size=None):
        input = input.permute(1, 0, 2)
        # Forward pass through LSTM layer
        # uncomment to switch to LSTM
        # lstm_out, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        gru_out, final_hidden_state = self.gru(input)

        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.softmax(self.linear2(self.dropout(self.sigmoid(self.linear(final_hidden_state[-1])))))
        return y_pred


class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers=1):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        # LSTM Layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, num_layers=num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, 128)
        self.linear2 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout()

    def init_states(self, batch_size):
        # This is what we'll initialise our hidden state as
        return Variable(torch.zeros(1, batch_size, self.hidden_dim)), Variable(torch.zeros(1, batch_size, self.hidden_dim))

    def forward(self, input, batch_size=None):
        input = input.permute(1, 0, 2)
        # Forward pass through LSTM layer
        h0, c0 = self.init_states(batch_size)
        lstm_out, (final_hidden_state, final_cell_state) = self.lstm(input, (h0, c0))

        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.softmax(self.linear2(self.dropout(self.sigmoid(self.linear(final_hidden_state[-1])))))
        return y_pred

class OldTwoCellGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim):
        super(OldTwoCellGRU, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        # LSTM Layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        
        # GRU Layer
        self.gru = nn.GRU(self.input_dim, self.hidden_dim)
        self.gru2 = nn.GRU(self.hidden_dim, self.hidden_dim)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, 128)
        self.linear2 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout()

    def init_states(self, batch_size):
        # This is what we'll initialise our hidden state as
        return Variable(torch.zeros(1, batch_size, self.hidden_dim)), Variable(torch.zeros(1, batch_size, self.hidden_dim))

    def forward(self, input, batch_size=None):
        input = input.permute(1, 0, 2)
        # Forward pass through LSTM layer
        # uncomment to switch to LSTM
        # lstm_out, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        gru_out, hidden_state = self.gru(input)
        gru_out, final_hidden_state = self.gru2(hidden_state)

        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.softmax(self.linear2(self.dropout(self.sigmoid(self.linear(final_hidden_state[-1])))))
        return y_pred


import numpy as np
import torch.nn.functional as F
from torch.nn import init


class ConvGRUCell(nn.Module):
    """
    Generate a convolutional GRU cell
    """

    def __init__(self, input_size, hidden_size, kernel_size):
        super().__init__()
        padding = kernel_size // 2
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.reset_gate = nn.Conv1d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)
        self.update_gate = nn.Conv1d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)
        self.out_gate = nn.Conv1d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)

        init.orthogonal(self.reset_gate.weight)
        init.orthogonal(self.update_gate.weight)
        init.orthogonal(self.out_gate.weight)
        init.constant(self.reset_gate.bias, 0.)
        init.constant(self.update_gate.bias, 0.)
        init.constant(self.out_gate.bias, 0.)


    def forward(self, input_, prev_state):

        # get batch and spatial sizes
        batch_size = input_.data.size()[0]
        spatial_size = input_.data.size()[2:]

        # generate empty prev_state, if None is provided
        if prev_state is None:
            state_size = [batch_size, self.hidden_size] + list(spatial_size)
            if torch.cuda.is_available():
                prev_state = Variable(torch.zeros(state_size)).cuda()
            else:
                prev_state = Variable(torch.zeros(state_size))

        # data size is [batch, channel, height, width]
        stacked_inputs = torch.cat([input_, prev_state], dim=1)
        update = F.sigmoid(self.update_gate(stacked_inputs))
        reset = F.sigmoid(self.reset_gate(stacked_inputs))
        out_inputs = F.tanh(self.out_gate(torch.cat([input_, prev_state * reset], dim=1)))
        new_state = prev_state * (1 - update) + out_inputs * update

        return new_state


class ConvGRU(nn.Module):

    def __init__(self, input_size, hidden_sizes, kernel_sizes, n_layers):
        '''
        Generates a multi-layer convolutional GRU.
        Preserves spatial dimensions across cells, only altering depth.
        Parameters
        ----------
        input_size : integer. depth dimension of input tensors.
        hidden_sizes : integer or list. depth dimensions of hidden state.
            if integer, the same hidden size is used for all cells.
        kernel_sizes : integer or list. sizes of Conv2d gate kernels.
            if integer, the same kernel size is used for all cells.
        n_layers : integer. number of chained `ConvGRUCell`.
        '''

        super(ConvGRU, self).__init__()

        self.input_size = input_size

        if type(hidden_sizes) != list:
            self.hidden_sizes = [hidden_sizes]*n_layers
        else:
            assert len(hidden_sizes) == n_layers, '`hidden_sizes` must have the same length as n_layers'
            self.hidden_sizes = hidden_sizes
        if type(kernel_sizes) != list:
            self.kernel_sizes = [kernel_sizes]*n_layers
        else:
            assert len(kernel_sizes) == n_layers, '`kernel_sizes` must have the same length as n_layers'
            self.kernel_sizes = kernel_sizes

        self.n_layers = n_layers

        cells = []
        for i in range(self.n_layers):
            if i == 0:
                input_dim = self.input_size
            else:
                input_dim = self.hidden_sizes[i-1]

            cell = ConvGRUCell(input_dim, self.hidden_sizes[i], self.kernel_sizes[i])
            name = 'ConvGRUCell_' + str(i).zfill(2)

            setattr(self, name, cell)
            cells.append(getattr(self, name))

        self.cells = cells


    def forward(self, x, hidden=None):
        '''
        Parameters
        ----------
        x : 4D input tensor. (batch, channels, height, width).
        hidden : list of 4D hidden state representations. (batch, channels, height, width).
        Returns
        -------
        upd_hidden : 5D hidden representation. (layer, batch, channels, height, width).
        '''
        if not hidden:
            hidden = [None]*self.n_layers

        input_ = x

        upd_hidden = []

        for layer_idx in range(self.n_layers):
            cell = self.cells[layer_idx]
            cell_hidden = hidden[layer_idx]

            # pass through layer
            upd_cell_hidden = cell(input_, cell_hidden)
            upd_hidden.append(upd_cell_hidden)
            # update input_ to the last updated hidden layer for next pass
            input_ = upd_cell_hidden

        # retain tensors in list to allow different hidden sizes
        return upd_hidden

In [ ]:
# NOT REQUIRED IF ALREADY SAVED NPY

dev_key_path = os.path.join("/content", "tc4tl", "docs", "tc4tl_dev_key.tsv")
test_key_path = os.path.join("/content", "tc4tl", "docs", "tc4tl_test_metadata.tsv")
dev_data_path = os.path.join("/content", "tc4tl", "data", "dev")
test_data_path = os.path.join("/content", "tc4tl", "data", "test")
dev_X, dev_y, labels_to_distance, interval_to_file = load_data(dev_key_path, dev_data_path)
test_X, _, test_labels_to_distance, test_intervals_to_file = load_data(test_key_path, test_data_path)
print(labels_to_distance)
print(test_labels_to_distance)

FileNotFoundError: ignored

In [ ]:
print(test_intervals_to_file)

['aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aadmezin_tc4tl20.csv', 'aaidcbne_tc4tl

In [ ]:
# NOT REQUIRED IF NPY SAVED ALREADY
EXP_NAME = ""
train_X, val_X, train_y, val_y = train_test_split(dev_X, dev_y, test_size=.1, random_state=100)
train_intervals_to_file, val_intervals_to_file = train_test_split(interval_to_file, test_size=.1, random_state=100)
np.save("/content/gdrive/My Drive/dev_X_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in dev_X])
np.save("/content/gdrive/My Drive/dev_y_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in dev_y])
np.save("/content/gdrive/My Drive/train_X_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in train_X])
np.save("/content/gdrive/My Drive/train_y_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in train_y])
np.save("/content/gdrive/My Drive/val_X_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in val_X])
np.save("/content/gdrive/My Drive/val_y_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in val_y])
np.save("/content/gdrive/My Drive/train_file_names_numpy.npy", np.array(train_intervals_to_file))
np.save("/content/gdrive/My Drive/val_file_names_numpy.npy", np.array(val_intervals_to_file))
np.save("/content/gdrive/My Drive/test_X_{}_numpy.npy".format(EXP_NAME), [tensor.numpy() for tensor in test_X])
np.save("/content/gdrive/My Drive/test_file_names_numpy.npy", np.array(test_intervals_to_file))

NameError: ignored

In [ ]:
# RUN TO LOAD SAVED NPY DATA
EXP_NAME = ""

import numpy as np
def generate_synthetic_hybrids(X, y, num_new_samples=1000):
  orig_X = X.copy()
  orig_y = y.copy()
  dataset_size = len(orig_y)
  for i in range(num_new_samples):
    a_index = random.randint(0, dataset_size - 1)
    a_y = orig_y[a_index]
    a_X = orig_X[a_index]
    b_index = random.randint(0, dataset_size - 1)
    b_y = orig_y[b_index]
    while (int(sum((b_y + a_y)[1:3])) % 2 != 0) or torch.equal(b_y, a_y):
      b_index = random.randint(0, dataset_size - 1)
      b_y = orig_y[b_index]
    b_X = orig_X[b_index]
    lam = np.random.beta(0.2, 0.2)
    new_X = lam * a_X + (1 - lam) * b_X
    new_y = lam * a_y + (1 - lam) * b_y
    X.append(new_X)
    y.append(new_y)
  return X, y

with open("/content/gdrive/My Drive/train_X_numpy.npy", 'rb') as f:
  train_X = [torch.from_numpy(el) for el in np.load(f)]
with open("/content/gdrive/My Drive/test_X_numpy.npy", 'rb') as f:
  test_X = [torch.from_numpy(el) for el in np.load(f)]
# with open("/content/gdrive/My Drive/dev_X_numpy.npy", 'rb') as f:
#   dev_X = [torch.from_numpy(el) for el in np.load(f)]
# with open("/content/gdrive/My Drive/dev_y_numpy.npy", 'rb') as f:
#   dev_y = [torch.from_numpy(el) for el in np.load(f)]
with open("/content/gdrive/My Drive/train_y_numpy.npy", 'rb') as f:
  train_y = [torch.from_numpy(el) for el in np.load(f)]
with open("/content/gdrive/My Drive/val_X_numpy.npy", 'rb') as f:
  val_X = [torch.from_numpy(el) for el in np.load(f)]
with open("/content/gdrive/My Drive/val_y_numpy.npy", 'rb') as f:
  val_y = [torch.from_numpy(el) for el in np.load(f)]
with open("/content/gdrive/My Drive/train_file_names_numpy.npy", 'rb') as f:
  train_intervals_to_file = np.load(f)
with open("/content/gdrive/My Drive/val_file_names_numpy.npy", 'rb') as f:
  val_intervals_to_file = np.load(f)
with open("/content/gdrive/My Drive/test_file_names_numpy.npy", 'rb') as f:
  test_intervals_to_file = np.load(f)

dev_X = train_X + val_X
dev_y = train_y + val_y

train_data_loader = DataLoader(list(zip(train_X, train_y)), batch_size=25)
val_data_loader = DataLoader(list(zip(val_X, val_y)), batch_size=25)
dev_data_loader = DataLoader(list(zip(dev_X, dev_y)), batch_size=25)
print(len(train_y))
# train_X, train_y = generate_synthetic_hybrids(train_X, train_y, num_new_samples=4000)
# print(len(train_y))

6042


In [ ]:

EXP_NAME = "CONV_GRU"
labels_to_distance = [1.2, 3.0, 4.5, 1.8]

MODEL_PATH = "/content/gdrive/My Drive/layers_1_epochs_200_{}".format(EXP_NAME)

model = ConvGRU(len(dev_X[0][0]), 200, output_dim=len(labels_to_distance), num_layers=1)
best_acc = 0
total_loss = []  # for plotting
loss_fn = CSELoss
train_start_time = time.time()
for epoch in range(200):
    total_epoch_loss = 0
    total_epoch_acc = 0
    total_epoch_tc_acc = 0
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    steps = 0
    model.train()
    for idx, batch in enumerate(train_data_loader):
        input = batch[0]
        label = torch.max(batch[1], axis=1)[1]
        optim.zero_grad()
        prediction = model(input)  batch[1])
        prediction = torch.max(prediction, 1)[1].view(label.size())
        num_corrects = (prediction == label).float().sum()
        acc = 100.0 * num_corrects / len(label)
        prediction = ((prediction == 0) + (prediction == 3).float())
        label = ((label == 0) + (label == 3).float())
        num_tc_corrects = (label == prediction).float().sum()
        tc_acc = 100.0 * num_tc_corrects / len(label)
        loss.backward()
        optim.step()
        steps += 1
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        total_epoch_tc_acc += tc_acc.item()
    if epoch % 1 == 0:
      # eval
      loss, acc, tc_acc = eval_model(model, val_data_loader)
      print("Testing Loss: {}, AVG: {}, TC4TL or Not AVG: {}".format(loss, acc, tc_acc))
    total_loss.append((epoch, total_epoch_loss/steps))
    print (f'Epoch: {epoch+1}, Training Loss: {total_epoch_loss/steps:.4f}, Training Accuracy: {total_epoch_acc/steps: .2f}% TC4TL or Not ACC: {total_epoch_tc_acc/steps: .2f}%')
print("finished training, took {} seconds".format(time.time() - train_start_time))
plt.scatter(*zip(*total_loss))
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.show()

# eval
loss, acc, tc_acc = eval_model(model, val_data_loader)
print("Testing Loss: {}, AVG: {}, TC4TL or Not AVG: {}".format(loss, acc, tc_acc))

TypeError: ignored

In [ ]:
# TEST ON VAL SECTION OF DEV DATA
val_data_loader = DataLoader(list(zip(val_X, val_y)), batch_size=25)
model2 = GRU(len(dev_X[0][0]), 200, batch_size=train_data_loader.batch_size, output_dim=len(labels_to_distance))
model2.load_state_dict(torch.load(MODEL_PATH))
loss, acc, tc_acc = eval_model(model2, val_data_loader)
print("Testing Loss: {}, AVG: {}, TC4TL or Not AVG: {}".format(loss, acc, tc_acc))

NameError: ignored

In [ ]:
# OUTPUT SAVE TO FILE
labels_to_distance = [1.2, 3.0, 4.5, 1.8]
MODEL_PATH = "/content/gdrive/My Drive/layers_1_epochs_200"
model2 = GRU(len(dev_X[0][0]), 200, batch_size=25, output_dim=len(labels_to_distance), num_layers=1)
print(sum(p.numel() for p in model2.parameters()))
model2.load_state_dict(torch.load(MODEL_PATH))
output_predictions(model2, test_X, labels_to_distance, test_intervals_to_file, "/content/gdrive/My Drive/1_layer_final_output_TEST.tsv")

161844
['aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaafrwlt_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aaasqzop_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aabadzsd_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacevzem_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aacjnxug_tc4tl20.csv', 'aadmezin_tc4tl20.csv', 'aaidcbn

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Streaming output truncated to the last 5000 lines.
52430
52431
52432
52433
52434
52435
52436
52437
52438
52439
52440
52441
52442
52443
52444
52445
52446
52447
52448
52449
52450
52451
52452
52453
52454
52455
52456
52457
52458
52459
52460
52461
52462
52463
52464
52465
52466
52467
52468
52469
52470
52471
52472
52473
52474
52475
52476
52477
52478
52479
52480
52481
52482
52483
52484
52485
52486
52487
52488
52489
52490
52491
52492
52493
52494
52495
52496
52497
52498
52499
52500
52501
52502
52503
52504
52505
52506
52507
52508
52509
52510
52511
52512
52513
52514
52515
52516
52517
52518
52519
52520
52521
52522
52523
52524
52525
52526
52527
52528
52529
52530
52531
52532
52533
52534
52535
52536
52537
52538
52539
52540
52541
52542
52543
52544
52545
52546
52547
52548
52549
52550
52551
52552
52553
52554
52555
52556
52557
52558
52559
52560
52561
52562
52563
52564
52565
52566
52567
52568
52569
52570
52571
52572
52573
52574
52575
52576
52577
52578
52579
52580
52581
52582
52583
52584
52585
52586
52587
5